In [49]:
import tensorflow as tf
import torch
!pip install datasets
!pip install transformers
!pip install sentencepiece

In [50]:
!nvcc --version
!nvidia-smi
print("PyTorch Version: ", torch.__version__)
print("PyTorch Cuda Version: ", torch.version.cuda)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
Wed May  5 14:08:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |         

In [51]:
from transformers import Trainer, TrainingArguments
from transformers import XLMRobertaTokenizer
select_model = 'xlm-roberta-base'   # xlm-roberta-large
tokenizer = XLMRobertaTokenizer.from_pretrained(select_model)

In [52]:
from datasets import load_dataset
path_to_load = '/content/en-train.json'
train_dataset, val_dataset = load_dataset('json', data_files=path_to_load, split=['train[:80%]','train[80%:]'])
train_dataset = train_dataset.rename_column('text','sentence')
val_dataset = val_dataset.rename_column('text','sentence')

Using custom data configuration default-af1d5f96301248e3
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-af1d5f96301248e3/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02)


In [53]:
from pprint import pprint
pprint(train_dataset.info.__dict__)

{'builder_name': 'json',
 'citation': '',
 'config_name': 'default',
 'dataset_size': 10180325,
 'description': '',
 'download_checksums': {'/content/en-train.json': {'checksum': '85ac02e6d5df94ccdbcfc47a987087ca21c646fcce9915d0d2e519d42952f72e',
                                                   'num_bytes': 10366199}},
 'download_size': 10366199,
 'features': {'id': Value(dtype='int64', id=None),
              'label': Value(dtype='int64', id=None),
              'sentence': Value(dtype='string', id=None)},
 'homepage': '',
 'license': '',
 'post_processed': None,
 'post_processing_size': None,
 'size_in_bytes': 20546524,
 'splits': {'train': SplitInfo(name='train', num_bytes=10180325, num_examples=9324, dataset_name='json')},
 'supervised_keys': None,
 'version': 0.0.0}


In [54]:
# Set voacb size
sentence_lenght = 200

# Check embeddings
batch_sentences = train_dataset['sentence'][:3]
print(batch_sentences)
encoded = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors='pt', max_length=sentence_lenght)
print(encoded)

print()
for ids in encoded["input_ids"]:
    pprint(tokenizer.decode(ids))


# Build embeddings on vocab
def tokenize(batch):
    return tokenizer(batch['sentence'], padding=True, truncation=True, max_length=sentence_lenght)  # batch['text']


train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
#test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
#test_dataset.set_format('torch', columns=['input_ids', 'attention_mask'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

["Schools 'need to adopt green approach'\nPUBLISHED : Monday, 08 March, 1999, 12:00am\nYou are never too young to learn to look after the environment, an official says.\nSpeaking at a seminar on environmental studies held at the Open University of Hong Kong, Mabel Mak Lok-sheung, senior environmental protection officer at the Environmental Protection Department, said concern for the environment should be taught from an early age.\nSchools should adopt a green approach to help create a better living environment, Ms Mak said.\nEnvironmental awareness would penetrate every level of society if children learned it at school, she said.\n'It should be a top-down approach. The first level is the management level.\n'Principals and administrative staff at schools should set an example for teachers ", 'IN SCHOOL\nKerber, defending Graf’s record, gets the message\nJanuary 29, 2016 00:00 IST\nAngelique Kerber\nI think she [Serena Williams] ', 'Microsoft launches Xbox Music on Web for free\nPUBLISHE

In [124]:
labels = train_dataset['label']
input_ids = train_dataset['input_ids']
masks = train_dataset['attention_mask']

In [125]:
X = input_ids * masks
Y = labels

In [126]:
X = X.type(torch.FloatTensor)
Y = Y.type(torch.LongTensor)

In [127]:
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(200,200)
        self.fc2 = nn.Linear(200,200)
        self.fc3 = nn.Linear(200,2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()

In [128]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [129]:
for epoch in range(1000):  # loop over the dataset multiple times
  i=0
  # zero the parameter gradients
  optimizer.zero_grad()

  # forward + backward + optimize
  outputs = net(X)
  loss = criterion(outputs, Y)
  loss.backward()
  optimizer.step()

  # print statistics
  if epoch%100 == 0:
    running_loss = loss.item()
    print('[%d] loss: %.3f' %
    (epoch + 1, running_loss))

print('Finished Training')

[1] loss: 1122.821
[101] loss: 0.507
[201] loss: 0.507
[301] loss: 0.507
[401] loss: 0.507
[501] loss: 0.507
[601] loss: 0.507
[701] loss: 0.507
[801] loss: 0.507
[901] loss: 0.507
Finished Training


In [130]:
_, predicted = torch.max(net(X),axis = 1)

In [135]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_curve
print(accuracy_score(Y, predicted))
print(f1_score(Y, predicted))
print(precision_score(Y, predicted))
print(recall_score(Y, predicted))


0.795280868749162
0.0
0.0
0.0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
